<h3>Name: Hassan Magdy Abdallah Nassar</h3>
<h3> Group:4</h3>

Let's start with your project: 

Are you a data scientist? 

I think you are an awesome a data scientist.

### **Problem** 
**Our goal is to create a predictive model that can answer the following question:**

**What kind of people had a better chance of surviving?**

**Data about passengers:**
*   Name
*   Age
*   Gender.


## Build Spark Session

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import findspark
findspark.init()
spark = SparkSession.builder.appName('stock').getOrCreate()
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Data Loading


You have two datasets: 
* Train  
* Test.

Read two datasets: 
* Train
* Test.



In [2]:
Train = spark.read.csv('train.csv', inferSchema=True, header=True)
Test = spark.read.csv('test.csv', inferSchema=True, header=True)

Let's work with train dataset:

**Confirm if this is a dataframe or not:**

In [3]:
type(Train)

pyspark.sql.dataframe.DataFrame

In [4]:
Test

DataFrame[PassengerId: int, Survived: int, Pclass: int, Name: string, Sex: string, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: string, Embarked: string]

**Show 5 rows.**

In [5]:
Train.show(5, truncate=True)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

**Display schema for the dataset:**

In [6]:
Train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



**Statistical summary:**

In [7]:
Train.summary().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|   Sex|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

## EDA - Exploratory Data Analysis

**Display count for the train dataset:**

In [8]:
Train.count()

891

**Can you answer this question:** 

**How many people survived, and how many didn't survive?** 

**Please save data in a variable.**

In [9]:
survived = Train.select('Survived').groupBy('Survived').count()

**Display your result:**

In [10]:
survived.show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



**Can you display your answer in ratio form?(Hint: Use UDF.)**

In [11]:
@udf(returnType=FloatType()) 
def ratio_df(c):
    return Train.select(c).where(col(c)==1).count()/Train.select(c).count()*100

In [12]:
c= 'Survived'
Train.select(c).where(col(c)==1).count()/Train.select(c).count()*100

38.38383838383838

**Can you get the number of males and females?**


In [13]:
g1 = Train.select('Sex').where(col('Sex').isNotNull()).groupBy('Sex').count()

g1.show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  314|
|  male|  577|
+------+-----+



In [14]:
Train.createOrReplaceTempView("Train")

**1. What is the average number of survivors of each gender?**

**2. What is the number of survivors of each gender?**

(Hint: Group by the "sex" column.)

In [15]:
spark.sql("""SELECT Sex,COUNT(Survived)/(SELECT COUNT(*) FROM Train) AS RATIO
             FROM Train
             GROUP BY Sex;""").show()

+------+-------------------+
|   Sex|              RATIO|
+------+-------------------+
|female|0.35241301907968575|
|  male| 0.6475869809203143|
+------+-------------------+



In [16]:
count = (Train.select(['Sex','Survived'])
 .where(col('Survived')==1)
 .groupBy('Sex')
 .count()
)
count.show()

+------+-----+
|   Sex|count|
+------+-----+
|female|  233|
|  male|  109|
+------+-----+



**Create temporary view PySpark:**

In [17]:
Train.createOrReplaceTempView("Train")

In [18]:
spark = SparkSession.builder.appName('SparkSQL').enableHiveSupport().getOrCreate()

**How many people survived, and how many didn't survive? By SQL:**

In [19]:
spark.sql("""SELECT SUM(Survived)as SURVIVED,COUNT(Survived)-SUM(Survived) AS NOT_SURVIVED FROM Train;""").show()

+--------+------------+
|SURVIVED|NOT_SURVIVED|
+--------+------------+
|     342|         549|
+--------+------------+



**Can you display the number of survivors from each gender as a ratio?**

(Hint: Group by "sex" column.)

**Can you do this via SQL?**

In [20]:
spark.sql("""SELECT Sex, SUM(Survived)as SURVIVED,
             COUNT(Survived)-SUM(Survived) AS NOT_SURVIVED 
             FROM Train
             GROUP BY Sex;""").show()

+------+--------+------------+
|   Sex|SURVIVED|NOT_SURVIVED|
+------+--------+------------+
|female|     233|          81|
|  male|     109|         468|
+------+--------+------------+



**Display a ratio for p-class:**


In [21]:
spark.sql("""SELECT Pclass,COUNT(Pclass)/(SELECT COUNT(*) FROM Train) AS RATIO
             FROM Train
             GROUP BY Pclass;""").show()

+------+-------------------+
|Pclass|              RATIO|
+------+-------------------+
|     1|0.24242424242424243|
|     3| 0.5510662177328844|
|     2|0.20650953984287318|
+------+-------------------+



**Let's take a break and continue after this.**

## Data Cleaning

**First and foremost, we must merge both the train and test datasets. (Hint: The union function can do this.)**



In [22]:
combined = Train.union(Test)
combined.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

**Display count:**

In [23]:
combined.count()

1329

**Temporary view PySpark:**

In [24]:
combined.createOrReplaceTempView('combined')

**Can you define the number of null values in each column?**


In [25]:
def null_value_count(df):
    null_columns_counts = []
    numRows = df.count()
    for k in df.columns:
        nullRows = df.where(col(k).isNull()).count()
        temp = k,nullRows
        null_columns_counts.append(temp)
    return(null_columns_counts)
null_value_count(combined)
spark.createDataFrame(null_value_count(combined)).show()

+-----------+----+
|         _1|  _2|
+-----------+----+
|PassengerId|   0|
|   Survived|   0|
|     Pclass|   0|
|       Name|   0|
|        Sex|   0|
|        Age| 265|
|      SibSp|   0|
|      Parch|   0|
|     Ticket|   0|
|       Fare|   0|
|      Cabin|1021|
|   Embarked|   3|
+-----------+----+



**Create Dataframe for null values**

1. Column
2. Number of missing values.

In [26]:
def null_value_count(df):
    null_columns_counts = []
    numRows = df.count()
    for k in df.columns:
        nullRows = df.where(col(k).isNull()).count()
        if(nullRows > 0):
            temp = k,nullRows
            null_columns_counts.append(temp)
    return(null_columns_counts)
null_value_count(combined)
spark.createDataFrame(null_value_count(combined), ['Column_With_Null_Value', 'Null_Values_Count']).show()

+----------------------+-----------------+
|Column_With_Null_Value|Null_Values_Count|
+----------------------+-----------------+
|                   Age|              265|
|                 Cabin|             1021|
|              Embarked|                3|
+----------------------+-----------------+



## Preprocessing 

**Can you show me the name column from your temporary table?**

In [27]:
combined.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

**Run this code:**

In [28]:
combined = combined.withColumn('Title',regexp_extract(col("Name"),"([A-Za-z]+)\.",1))
combined.createOrReplaceTempView('combined')

**Display the title and count "Title" column:**

In [29]:
c = combined.select('title').groupBy('title').count()
c.show()

+--------+-----+
|   title|count|
+--------+-----+
|     Don|    1|
|    Miss|  257|
|Countess|    2|
|     Col|    4|
|     Rev|    9|
|    Lady|    2|
|  Master|   56|
|     Mme|    1|
|    Capt|    2|
|      Mr|  786|
|      Dr|   11|
|     Mrs|  186|
|     Sir|    2|
|Jonkheer|    2|
|    Mlle|    4|
|   Major|    3|
|      Ms|    1|
+--------+-----+



**We can see that Dr, Rev, Major, Col, Mlle, Capt, Don, Jonkheer, Countess, Ms, Sir, Lady, and Mme are really rare titles, so create Dictionary and set the value to "rare".**

In [30]:
titles_map = {'Dr':'rare', 'Rev':'rare', 'Major':'rare', 'Col':'rare',
              'Mlle':'rare', 'Capt':'rare', 'Don':'rare', 'Jonkheer':'rare',
              'Countess':'rare', 'Ms':'rare', 'Sir':'rare', 'Lady':'rare',
              'Mme':'rare','Miss':'not_rare','Master':'not_rare','Mr':'not_rare','Mrs':'not_rare'}

**Run the function:**

In [31]:
def impute_title(title):
    return titles_map[title]

**Apply the function on "Title" column using UDF:**

In [32]:
c = c.withColumn("count",when((c['count']<50), 'rare').otherwise(c["count"]))

In [33]:
convertUDF = udf(lambda z: impute_title(z))
#c.select(convertUDF(col("title")).alias("Title") ).show(truncate=False)

**Display "Title" from table and group by "Title" column:**

In [34]:
c.show()

+--------+-----+
|   title|count|
+--------+-----+
|     Don| rare|
|    Miss|  257|
|Countess| rare|
|     Col| rare|
|     Rev| rare|
|    Lady| rare|
|  Master|   56|
|     Mme| rare|
|    Capt| rare|
|      Mr|  786|
|      Dr| rare|
|     Mrs|  186|
|     Sir| rare|
|Jonkheer| rare|
|    Mlle| rare|
|   Major| rare|
|      Ms| rare|
+--------+-----+



## **Preprocessing Age**

**Based on the age mean, you will fill in the missing age values:**

In [35]:
mean = int(combined.groupBy().avg('Age').collect()[0][0])
mean

30

**Fill missing age with age mean:**

In [36]:
combined = combined.withColumn("Age",when((combined["Age"].isNull()), mean).otherwise(combined["Age"]))

## **Preprocessing Embarked**

**Select Embarked, count them, order by count Desc, and save in grouped_Embarked variable:**




In [37]:
grouped_Embarked = combined.groupBy("Embarked").count().sort(col("count").desc())

**Show groupped_Embarked:**

In [38]:
grouped_Embarked.show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  962|
|       C|  253|
|       Q|  111|
|    null|    3|
+--------+-----+



**Get the groupped_Embarked:** 

In [39]:
top = grouped_Embarked.first()[0]

In [40]:
top

'S'

**Fill missing values with grouped_Embarked:**

In [41]:
combined = combined.withColumn("Embarked",when((combined["Embarked"].isNull()), top).otherwise(combined["Embarked"]))

In [42]:
combined.groupBy("Embarked").count().sort(col("count").desc()).show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  965|
|       C|  253|
|       Q|  111|
+--------+-----+



## **Preprocessing Cabin**

**Replace "cabin" column with first char from the string:**



**Show the result:**

In [43]:
combined = combined.withColumn("Cabin",when(~(combined["Cabin"].isNull()), combined["Cabin"].substr(1, 1)).otherwise(combined["Cabin"]))

In [44]:
combined.select('Cabin').show()

+-----+
|Cabin|
+-----+
| null|
|    C|
| null|
|    C|
| null|
| null|
|    E|
| null|
| null|
| null|
|    G|
|    C|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
| null|
+-----+
only showing top 20 rows



**Create the temporary view:**

In [45]:
combined.createOrReplaceTempView('combined')

**Select "Cabin" column, count Cabin column, Group by "Cabin" column, Order By count DESC**  

In [46]:
combined.groupBy("Cabin").count().sort(col("count").desc()).show()

+-----+-----+
|Cabin|count|
+-----+-----+
| null| 1021|
|    C|   82|
|    B|   77|
|    D|   52|
|    E|   51|
|    A|   23|
|    F|   18|
|    G|    4|
|    T|    1|
+-----+-----+



**Fill missing values with "U":**

In [47]:
combined = combined.withColumn("Cabin",when((combined["Cabin"].isNull()), 'U').otherwise(combined["Embarked"]))

In [48]:
combined.groupBy("Cabin").count().sort(col("count").desc()).show()

+-----+-----+
|Cabin|count|
+-----+-----+
|    U| 1021|
|    S|  199|
|    C|  104|
|    Q|    5|
+-----+-----+



**StringIndexer: A label indexer that maps a string column of labels to an ML column of label indices. If the input column is numeric, we cast it to string and index the string values. The indices are in [0, numLabels). By default, this is ordered by label frequencies so the most frequent label gets index 0. The ordering behavior is controlled by setting stringOrderType. Its default value is ‘frequencyDesc’.**

**StringIndexer(inputCol=None, outputCol=None)**

**Pipeline: ML Pipelines provide a uniform set of high-level APIs built on top of DataFrames that help users create and tune practical machine learning pipelines.**

____________________________________________

**Use Pipline to fit and transform:**

In [49]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, OneHotEncoder, VectorAssembler, IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [50]:
genderIndexer = StringIndexer(inputCol="Sex", outputCol="indexedSex")
embarkIndexer = StringIndexer(inputCol="Embarked", outputCol="indexedEmbarked")
 
surviveIndexer = StringIndexer(inputCol="Survived", outputCol="indexedSurvived")
 
genderEncoder = OneHotEncoder(inputCol="indexedSex", outputCol="sexVec")
embarkEncoder = OneHotEncoder(inputCol="indexedEmbarked", outputCol="embarkedVec")
 
assembler = VectorAssembler(inputCols=["Pclass","sexVec","Age","SibSp","Fare","embarkedVec"],outputCol="features")
 
rf = RandomForestClassifier(labelCol="indexedSurvived", featuresCol="features")
 
pipeline = Pipeline(stages=[surviveIndexer, genderIndexer,
                            embarkIndexer, genderEncoder,embarkEncoder, assembler, rf])

**VectorAssembler: VectorAssembler(*, inputCols=None, outputCol=None) A feature transformer that merges multiple columns into a vector column.**



**Use randomSplit function and split data to x_train, and X_test with 80% and 20% Consecutive**

In [51]:
(traindf, testdf) = combined.randomSplit([0.8,0.2])

**Build RandomForestClassifier model and use pipeline to fit and transform then display "prediction, Survived, features" columns**

In [52]:
model = pipeline.fit(traindf)

predictions = model.transform(testdf)
 
predictions.columns 
 
predictions.select("prediction", "Survived", "features").show(5)
 
predictions = predictions.select(col("Survived").cast("Float"),col("prediction"))
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
 
rfModel = model.stages[6]
print(rfModel)

+----------+--------+--------------------+
|prediction|Survived|            features|
+----------+--------+--------------------+
|       0.0|       1|[3.0,0.0,26.0,0.0...|
|       0.0|       0|[3.0,1.0,35.0,0.0...|
|       1.0|       1|[2.0,0.0,14.0,1.0...|
|       0.0|       1|[3.0,0.0,4.0,1.0,...|
|       0.0|       0|[3.0,1.0,39.0,1.0...|
+----------+--------+--------------------+
only showing top 5 rows

Test Error = 0.206897
RandomForestClassificationModel: uid=RandomForestClassifier_a841f59e3fee, numTrees=20, numClasses=2, numFeatures=7


**Use MulticlassClassificationEvaluator and set the "labelCol" to "Survived",  "predictionCol" to "prediction", "metricName" to "accuracy"** 

In [53]:
evaluator = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)
 
evaluatorf1 = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="f1")
f1 = evaluatorf1.evaluate(predictions)
print("f1 = %g" % f1)
 
evaluatorwp = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="weightedPrecision")
wp = evaluatorwp.evaluate(predictions)
print("weightedPrecision = %g" % wp)
 
evaluatorwr = MulticlassClassificationEvaluator(labelCol="Survived", predictionCol="prediction", metricName="weightedRecall")
wr = evaluatorwr.evaluate(predictions)
print("weightedRecall = %g" % wr)

Accuracy = 0.793103
f1 = 0.785481
weightedPrecision = 0.798528
weightedRecall = 0.793103


In [54]:
spark.stop()

**When you are finished send the project via Google classroom**
**Please let me know if you have any questions.**
* nabieh.mostafa@yahoo.com
* +201015197566 (Whatsapp)

**Don't Hate me, I push you to learn**

**I will help you to become an awesome data engineer.**

**Why did I say that "Data Engineer"?**

**Tricky question, but an optional question, if you would like to know the answer, ask me.**
